In [1]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
from collections import Counter

In [2]:
data = pd.read_csv('data/apcd_clean.csv')
data.head()

,البيت
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...


In [3]:
def score_sentence(sentence, keywords):
    words = sentence.split()
    word_counts = Counter(words)
    
    score = sum(word_counts[keyword] for keyword in keywords if keyword in word_counts)
    return {'score': score, 'sentence': sentence}

def parallel_score(data, keywords, num_processes=4):
    chunk_size = len(data) // num_processes
    chunks = [data.iloc[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
    
    with Pool(processes=num_processes) as pool:
        results = pool.starmap(score_chunk, [(chunk, keywords) for chunk in chunks])
    
    scored_sentences = [item for sublist in results for item in sublist]
    return scored_sentences

def score_chunk(chunk, keywords):
    return [score_sentence(sentence, keywords) for sentence in chunk['البيت']]

In [4]:
query = 'إذا حكم القضاء'

keywords = set(query.split())
scored_sentences = parallel_score(data, keywords)
df_results = pd.DataFrame(scored_sentences).sort_values('score', ascending=False)
df_results.head()

,score,sentence
1679,4,ثقال إذا لاقوا خفاف إذا دعوا كثير إذا شدوا ...
889490,4,قضيب إذا ما اهتز ظبي إذا رنا سنان إذا ما لا...
476749,4,سبوق إذا جارى لحوق إذا هوى غلوب إذا بادى قت...
1361063,4,سحاب إذا أعطى ونار إذا سطا وحبر إذا أفتى وب...
480081,4,صبح إذا سفرت غصن إذا خطرت ظبي إذا نفرت مسك ...


In [5]:
prompts = [
    'الحماسة والفخر',
    'روعة البحر',
    'الأمل والتفاؤل',
    'معاني الحياة والموت',
    'وجدانية الحب العذري',
    'التأمل في مصائر الإنسان',
    'حياة العرب قبل الإسلام',
    'العدمية والبحث عن الذات',
    'الفضيلة والأخلاق',
    'السلام والتسامح',
    'العدالة والمساواة',
    'كيف تخاف الفقر والله رازقا',
    'التشرد والهجرة والبحث عن الوطن',
    'عيناك نازلتاالقلوب',
    ' كتب الدمع بخدي عهده',
    'لا يحمل الحقد من تعلو به الرتب',
    'أتعرف رسم الدار',
    'الأصدقاء مثل النجوم',
    ' أحلك الأوقات',
    'شعر بدوي',
    ' الثقة بالله',
    ' ذكريات الطفولة',
    'المرء يأمل أن يعيش',
    'الوطن والانتماء',
    ' الحرية والنضال',
    'قم للمعلم وفه التبجيلا',
    'وعذرتهم وعرفت ذنبي',
    'إليك يا أمي',
    'فأصبح بعد الحرب',
    'إذا حكم القضاء',
    'ليت الأحزان',
    'فرحي مثل طائر',
    'نار الغضب',
    'صوت المطر ينعش روحي',
    'تنير ليالي',
    'إِن الثعالب بالضحى',
    'صبور على مر الحوادث',
    'البحث عن الحقيقة رحلة لا نهاية لها',
    'مرآة الروح',
    'كلما نسينا دروس الماضي',
    'دع الأيام تفعل ما تشاء',
    'وقفت على الديار',
    'رمال الصحراء',
    'شموخ',
    'تفاؤل',
    'نصر',
    'حمامة',
    'ساذج'
]

f = open("output_keyword.txt", 'w')

for prompt in prompts:
    f.write(prompt + "\n")
    
    keywords = set(prompt.split())
    scored_sentences = parallel_score(data, keywords)
    df_results = pd.DataFrame(scored_sentences).sort_values('score', ascending=False).reset_index()
    
    f.write(df_results.at[0, 'sentence'] + "\n")
    f.write(df_results.at[1, 'sentence'] + "\n")    
    f.write(df_results.at[2, 'sentence'] + "\n")
    
    f.write("\n")
f.close()